In [223]:
!pip install -U -q PyDrive

In [224]:
!pip install -U -q pyarrow

In [225]:
# importações necessarias
import pandas as pd
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import warnings
import shutil
warnings.filterwarnings('ignore')
import plotly.express as px
import pyarrow as pa
import pyarrow.parquet as pq

In [226]:
# Download dos dados para esta máquina virtual
shutil.unpack_archive(f"/content/drive/MyDrive/Pessoal/archive.zip", "./dataset")
#shutil.unpack_archive(f"/content/dataset/06120028.zip", "./dataset")
df_read = pd.read_csv('/content/dataset/Activity_history.csv')

In [227]:
df_read.isna().sum()

App name    1
Date        3
Time        3
Duration    3
dtype: int64

In [228]:
df_traino = df_read.dropna()

In [229]:
df_traino

,App name,Date,Time,Duration
0,App Usage,11/08/23,5:29:39 pm,0:00:01
1,WhatsApp,11/08/23,5:28:41 pm,0:00:57
2,WhatsApp,11/08/23,5:28:26 pm,0:00:10
3,App Usage,11/08/23,5:26:06 pm,0:02:19
4,Activity history enabled,11/08/23,5:26:06 pm,0:00:00
...,...,...,...,...
4007,WhatsApp,01/08/23,11:49:27 pm,0:00:05
4008,Google,01/08/23,11:49:25 pm,0:00:00
4009,Chrome,01/08/23,11:48:48 pm,0:00:37
4010,Google,01/08/23,11:48:44 pm,0:00:04


In [230]:
df_traino["App name"].unique()

array(['App Usage', 'WhatsApp', 'Activity history enabled',
       'Permission controller', 'Google Play Store', 'Swiggy',
       'Screen on (unlocked)', 'Screen off (locked)', 'Truecaller',
       'Phone', 'ICICIdirect Markets', 'Instagram', 'Gmail', 'YouTube',
       'Chrome', 'Google', 'Prime Video', 'Spotify', 'Maps', 'Calculator',
       'Manage notifications', 'Wireless Earphones', 'Drive', 'Messages',
       'Clock', 'Thunder VPN', 'LinkedIn', 'Lightroom', 'Snapchat',
       'Doc Scanner', 'Camera', 'Gymshark', 'Breeze', 'Screenshot',
       'Docs', 'Telegram', 'Photos', 'Paytm', 'McDelivery', "McDonald's",
       'MyJio', 'GPay', 'Fit', 'inshorts', 'X', 'Alien Shooter World',
       'Audible', 'Phone calls', 'Sweatcoin', 'Files', 'Ola', 'Calendar',
       'YT Studio', 'Amazon', 'Shazam', 'Google News', 'Key Chain',
       'Excel'], dtype=object)

In [231]:
colunas_interresse = [
    "WhatsApp", "Instagram", "Gmail", "YouTube"
]

In [232]:
novo_df = df_traino.groupby(['App name', 'Date'])['Duration'].sum().reset_index()

In [233]:
segundos = df_traino["Duration"]

In [234]:
from numba.core.types.scalars import Float
segundos = []
for time in df_traino["Duration"]:
  a = time.split(":")
  b = (int(a[0])*100000) + (int(a[1])*1000) + (int(a[2])*10)
  segundos.append(b)


In [235]:
df_traino["Duration"] = segundos

In [236]:
df_traino

,App name,Date,Time,Duration
0,App Usage,11/08/23,5:29:39 pm,10
1,WhatsApp,11/08/23,5:28:41 pm,570
2,WhatsApp,11/08/23,5:28:26 pm,100
3,App Usage,11/08/23,5:26:06 pm,2190
4,Activity history enabled,11/08/23,5:26:06 pm,0
...,...,...,...,...
4007,WhatsApp,01/08/23,11:49:27 pm,50
4008,Google,01/08/23,11:49:25 pm,0
4009,Chrome,01/08/23,11:48:48 pm,370
4010,Google,01/08/23,11:48:44 pm,40


In [237]:
df_filtrado = df_traino[df_traino['App name'].isin(colunas_interresse)]

In [238]:
df_traino = df_filtrado

In [239]:
from numba.core.types.scalars import Float
times = []
for time in df_traino["Time"]:
  a = time.split(":")
  a2 = a[2].split(" ")
  b = (int(a[0])*100000) + (int(a[1])*1000) + (int(a2[0])*10)
  times.append(b)

df_traino["Time"] = times

In [240]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

lb = LabelEncoder()


apps = lb.fit_transform(df_traino["App name"])
apps

array([2, 2, 1, ..., 2, 2, 2])

In [241]:
df_traino["labels_app"] = apps
df_traino

,App name,Date,Time,Duration,labels_app
1,WhatsApp,11/08/23,528410,570,2
2,WhatsApp,11/08/23,528260,100,2
31,Instagram,11/08/23,430120,19120,1
34,WhatsApp,11/08/23,423030,170,2
37,WhatsApp,11/08/23,416200,6120,2
...,...,...,...,...,...
4002,Instagram,01/08/23,1150110,1070,1
4003,WhatsApp,01/08/23,1150060,10,2
4006,WhatsApp,01/08/23,1149370,80,2
4007,WhatsApp,01/08/23,1149270,50,2


In [242]:
oneHot = OneHotEncoder()
arr = oneHot.fit_transform(df_traino[["labels_app"]]).toarray()
arr

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]])

In [243]:
one_hot_df = pd.DataFrame(arr, columns=[f'App_id_{i}' for i in range(arr.shape[1])])
one_hot_df

,App_id_0,App_id_1,App_id_2,App_id_3
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0
...,...,...,...,...
1112,0.0,1.0,0.0,0.0
1113,0.0,0.0,1.0,0.0
1114,0.0,0.0,1.0,0.0
1115,0.0,0.0,1.0,0.0


In [244]:
new_df = df = pd.concat([df_traino, one_hot_df], axis=1)
new_df = new_df.drop(["App name","labels_app","Date" ],axis=1)

In [245]:
new_df

,Time,Duration,App_id_0,App_id_1,App_id_2,App_id_3
1,528410.0,570.0,0.0,0.0,1.0,0.0
2,528260.0,100.0,0.0,1.0,0.0,0.0
31,430120.0,19120.0,0.0,0.0,1.0,0.0
34,423030.0,170.0,0.0,0.0,1.0,0.0
37,416200.0,6120.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...
1108,NaN,NaN,0.0,1.0,0.0,0.0
1109,NaN,NaN,0.0,1.0,0.0,0.0
1110,NaN,NaN,0.0,0.0,1.0,0.0
1111,NaN,NaN,0.0,0.0,1.0,0.0


In [246]:
from sklearn.preprocessing import MinMaxScaler

# Suponha que você tenha um DataFrame 'df' com características numéricas que você deseja escalar

# Crie uma instância do MinMaxScaler
scaler = MinMaxScaler()

# Ajuste e transforme os dados
df_scaled = scaler.fit_transform(new_df[["Time"]])

# O resultado é uma matriz NumPy, você pode convertê-la de volta em um DataFrame, se desejar
df_scaled = pd.DataFrame(df_scaled, columns=["Time"])

#new_df["Time"]=df_scaled

In [247]:
new_df=new_df.dropna()

In [248]:
new_df.head(100)

,Time,Duration,App_id_0,App_id_1,App_id_2,App_id_3
1,528410.0,570.0,0.0,0.0,1.0,0.0
2,528260.0,100.0,0.0,1.0,0.0,0.0
31,430120.0,19120.0,0.0,0.0,1.0,0.0
34,423030.0,170.0,0.0,0.0,1.0,0.0
37,416200.0,6120.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...
317,1018520.0,1430.0,0.0,0.0,1.0,0.0
318,1018340.0,180.0,0.0,0.0,0.0,1.0
319,1008200.0,10110.0,0.0,0.0,1.0,0.0
320,1008160.0,20.0,0.0,1.0,0.0,0.0


In [249]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Suponha que você tenha um DataFrame 'df' com suas características e rótulos de preço
X = new_df.drop('Duration', axis=1)  # Features
y = new_df['Duration']  # Target variable

# Divida os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crie um modelo de regressão linear
model = LinearRegression()

# Treine o modelo
model.fit(X_train, y_train)

# Faça previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avalie o modelo
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)

Mean Squared Error: 30462270.940398265
Root Mean Squared Error: 5519.263623020581
R-squared: 0.041246024235682355


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Avalie o modelo usando várias métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')


print("Acurácia:", accuracy)
print("Precisão:", precision)
print("Revocação:", recall)
print("F1-Score:", f1)


In [252]:

data = [
    [528410.0, 0.0, 0.0, 1.0, 0.0],

]

y_pred = model.predict(
    data
)
y_pred

array([2584.46634189])